## 환경설정

In [3]:
#pip install numpy
#pip install pandas
##한국어 자연어 처리 라이브러리
#pip install konlpy
#pip install nltk

#pip install tensorflow
#db 데이터 불러오기용
#!pip install import_ipynb

#파이썬 한글 맞춤법 검사 라이브러리
# pip install py-hanspell
#PyKoSpacing 설치
# pip install git+https://github.com/haven-jeon/PyKoSpacing.git


# shif+tab 키 활성화
%config Completer.use_jedi = False

## import library

In [4]:
import pandas as pd
import numpy as np
import nltk
import konlpy
from konlpy.tag import Okt, Mecab
import re 

from pykospacing import Spacing # 띄어쓰기
from hanspell import spell_checker # 맞춤법 (에민 -> 예민)

from collections import Counter
from tqdm import tqdm
import time


pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
pd.set_option('display.max_seq_items', None)

## DB 에 적재된 보호중인 동물의 성격데이터 불러오기 

In [10]:
# 해당 내용변수에 할당 
import import_ipynb
# from 뒤 파일명 주의 요함
from from_db_to_data import from_db_to_data
protecting_dog = from_db_to_data() #유기견 개체번호랑 특징 데이터만 추출


importing Jupyter notebook from from_db_to_data_3.ipynb


c:\Users\SooBin\.conda\envs\abandoned_dog\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\SooBin\.conda\envs\abandoned_dog\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [11]:
protecting_dog

,desertionNo,specialMark
0,426336202000787,강서구1-418호
1,429359202000091,20-0510 소두
2,429359202000295,"20-1101 녹수 임신의심,가죽목줄(주황) 착용"
3,429360202000026,20-0139 바니
4,429360202000191,20-0728 솜이
...,...,...
951,448567202001402,J-12-1-(2)-1 갈색+검정색.(J-11-1-6자견) CDV음성20/12/10
952,448567202001407,J-12-1-(2)-6 갈색+검정색.(J-11-1-6자견) CDV음성20/12/10
953,448567202001423,J-12-1-25 검정색+흰색. 가슴+발가락:흰색. 겁이 많음.
954,448567202001437,J-12-1-32-3 검정색+흰색


## 전처리

### 결측치 제거 (nulll값 있는 거 삭제)

In [12]:
# 결측치 개수 확인 후 결측치 있으면 제거
protecting_dog.isnull().sum()

desertionNo    0
specialMark    0
dtype: int64

In [13]:
# protecting_dog.dropna(inplace=True)
# # 결측지 제거시 확인용
# protecting_dog.isnull().sum()

-----

### 한글과 숫자 빼고 전부 제거

In [14]:
#한글, 숫자 빼고 전부 제거"""
def only_kor_num(sentence):
  return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣]','',str(sentence)) 

In [15]:
protecting_dog['specialMark'] = protecting_dog['specialMark'].apply(only_kor_num)
protecting_dog

,desertionNo,specialMark
0,426336202000787,강서구호
1,429359202000091,소두
2,429359202000295,녹수임신의심가죽목줄주황착용
3,429360202000026,바니
4,429360202000191,솜이
...,...,...
951,448567202001402,갈색검정색자견음성
952,448567202001407,갈색검정색자견음성
953,448567202001423,검정색흰색가슴발가락흰색겁이많음
954,448567202001437,검정색흰색


### 맞춤법(hanspell) 

In [16]:
marks = []
for mark in protecting_dog['specialMark']:
    spelled_sent = spell_checker.check(mark)
    hanspell_sent = spelled_sent.checked
    # marks.append(hanspell_sent)
    mark = hanspell_sent
print(protecting_dog['specialMark'])

0                               강서구호
1                                 소두
2                     녹수임신의심가죽목줄주황착용
3                                 바니
4                                 솜이
                   ...              
951                        갈색검정색자견음성
952                        갈색검정색자견음성
953                 검정색흰색가슴발가락흰색겁이많음
954                            검정색흰색
955    흰색연갈색쫑긋한귀꼬리끝남기고짧게미용되어있음줄무늬티착용
Name: specialMark, Length: 956, dtype: object


### 띄어쓰기(spacing)

In [17]:
spacing = Spacing()
protecting_dog['specialMark']= protecting_dog['specialMark'].astype(str).apply(lambda x : spacing(x))

### 성격에대한 데이터 라벨링

In [18]:
# 로컬에서의 성격 키워드 파일 불러와 번수에 저장 및 출력
personality = []
f = open('./dog_personality.txt','rt',encoding='utf-8') # 인코딩
lines = f.readlines()
for line in lines:
    line = line.strip()
    line = line.split(',')
    for word in line:
        personality.append(word)
f.close()
print(personality)

['순함', '온순', '성격', '사람', '겁', '경계', '적응', '소심', '기력', '얌전', '조용', '사나', '활발', '예민', '낯', '까칠', '영리', '호기심', '밝', '발랄', '장난', '조심', '애교', '의젓', '씩씩함']


In [19]:
# 성격 파일에 있는 단어가 있는 아이들은 True 처리, 없으면 False
words = '|'.join(personality)
test = protecting_dog.copy()
test['personality'] = test['specialMark'].str.contains(words)
test

,desertionNo,specialMark,personality
0,426336202000787,강서구호,False
1,429359202000091,소두,False
2,429359202000295,녹수임신의 심가죽목줄주황 착용,False
3,429360202000026,바니,False
4,429360202000191,솜이,False
...,...,...,...
951,448567202001402,갈색 검정색 자 견 음성,False
952,448567202001407,갈색 검정색 자 견 음성,False
953,448567202001423,검정색 흰색 가슴발가락 흰색 겁이 많음,True
954,448567202001437,검정색 흰색,False


In [20]:
# true 인 애들만 색출
true_p = (test.loc[test['personality']==True,:]).reset_index(drop=True)
true_p

,desertionNo,specialMark,personality
0,429360202000250,풍딸 순함,True
1,429361202000102,미예경계목줄 착용 빨강 생리중,True
2,429362202000574,용검 겁이 많음 공격형,True
3,429363202000370,크리순함 노란 목줄 착용,True
4,429363202000572,본 이 피부병 경계,True
...,...,...,...
385,448567202001370,갈색 검정색 사람을 잘 따르고 똥꼬 발랄함,True
386,448567202001372,갈색 흰색 사람을 잘 따르고 똥꼬 발랄함,True
387,448567202001373,회갈색 흰색 사람을 잘 따르고 똥꼬 발랄함,True
388,448567202001395,갈색 검정색 사람을 잘 따르고 순함 개월령 추정,True



### 성격 언급된 데이터 전처리 및 Okit 형태소분석

In [21]:
# 전처리 및 토큰화
sm_cols = true_p['specialMark']

for i, row in enumerate(sm_cols):
    if "경계심" in row:
        row = row.replace("경계심", "경계")
        sm_cols[i] = row

    if "심해" in row:
        row = row.replace("심해", "심하다")
        sm_cols[i] = row
okt = Okt()
true_p['tok_p'] = sm_cols.astype('str').apply(okt.pos,norm=True,stem=True)
true_p.head(20)

C:\Users\SooBin\AppData\Local\Temp\ipykernel_10948\2826358307.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sm_cols[i] = row
C:\Users\SooBin\AppData\Local\Temp\ipykernel_10948\2826358307.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sm_cols[i] = row


,desertionNo,specialMark,personality,tok_p
0,429360202000250,풍딸 순함,True,"[(풍, Adverb), (딸, Noun), (순하다, Adjective)]"
1,429361202000102,미예경계목줄 착용 빨강 생리중,True,"[(밉다, Adjective), (예, Modifier), (경계, Noun), (..."
2,429362202000574,용검 겁이 많음 공격형,True,"[(용검, Noun), (겁, Noun), (이, Josa), (많다, Adject..."
3,429363202000370,크리순함 노란 목줄 착용,True,"[(크리, Noun), (순하다, Adjective), (노랗다, Adjective..."
4,429363202000572,본 이 피부병 경계,True,"[(보다, Verb), (이, Noun), (피부병, Noun), (경계, Noun)]"
5,431369202000019,순함,True,"[(순하다, Adjective)]"
6,431369202000071,순함,True,"[(순하다, Adjective)]"
7,431370202000025,엄청 예쁘고 순함 개월 추정,True,"[(엄청, Adverb), (예쁘다, Adjective), (순하다, Adjecti..."
8,431370202000210,겁 많음 형제 마리입소,True,"[(겁, Noun), (많다, Adjective), (형제, Noun), (마리, ..."
9,431370202000811,코 부위 피부병 경계 있음,True,"[(코, Noun), (부위, Noun), (피부병, Noun), (경계, Noun..."


## 칼럼 매핑 및 수치화 작업 시작

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [26]:
#성격 분류
char_dict={'순함': ['순함','온순','착하다','순하다','조용','차분','의젓'],
           '소심':['겁','소심','소심하다'],
           '활발':['활발','발랄','밝다','장난','애교','씩씩'],
           '사나움':['까칠','경계','낯','짖음']}

# # 성격 나타난 것들만 카테고리 추출 
# dog_soon = test.loc[test['specialMark'].str.contains('순함|온순|착|조용|차분|의젓|순', regex=True, na=False)]
# dog_fear = test.loc[test['specialMark'].str.contains('겁|소심|무서|', regex=True, na=False)]
# dog_bright = test.loc[test['specialMark'].str.contains('활발|발랄|밝다|장난|애교|씩씩', regex=True, na=False)]
# dog_rough = test.loc[test['specialMark'].str.contains('사나|까칠|경계|낯', regex=True, na=False)]


,desertionNo,specialMark,personality
10,429361202000102,미예경계목줄 착용 빨강 생리중,True
36,429363202000572,본 이 피부병 경계,True
52,431370202000811,코 부위 피부병 경계심 있음,True
53,431370202000984,경계심심해 상 세 정보 추후 기재,True
54,431370202000985,경계심심해 상 세 정보 추후 기재,True
...,...,...,...
864,448532202000653,이브 겁 많음 경계심 있음 실물파라 외모만으론 현장에선 입양 문의 많음,True
870,448532202000758,버찌 출산 경험 많아 보임낯 가림순함,True
871,448532202000770,여울이 세 마리 동반입소보호소 내 유기낯 가림천천히 다가와서 냄새 맡아줌,True
872,448532202000773,웃을 이순함 약간 낯 가림,True


In [27]:
# word = '사나움'
def dog_char_related(marks):
    dog_char_related_list = char_dict[word]
    # specialMark가 word의 value중 하나라도 포함하면 1, 그렇지 않으면 0
    for dog_char_related_word in dog_char_related_list:
        if dog_char_related_word in marks:
            return 1
            #여기에 조건문 추가
    return 0

In [81]:
# 관련 단어 분류하는 함수 적용하기
for word in char_dict:
    true_p[word] = true_p['specialMark'].map(dog_char_related)

,desertionNo,specialMark,personality,dogrelated Word
360,448532202000791,소다년 월생추정 순함 사람 좋아함,True,[순함]
361,448532202000799,로망이 년월생추정자 매동 반입소순함,True,[순함]
362,448532202000800,초롱이 년월생 추정자 매동 반입소순함,True,[순함]
363,448532202000809,달자년월생추정 남매동 반입소순함,True,[순함]
364,448532202000810,소라년월생추정 남매동 반입소순함,True,[순함]
365,448532202000811,가을이년월생추정 남매동 반입소순함,True,[순함]
366,448532202000826,뽀뽀년 월생추정자매맑음이와 함께 입소 순함,True,[순함]
367,448532202000827,맑음이 년월생 추정자매뽀뽀와 함께 입소 순함,True,[순함]
368,448532202000843,레년월생 추정 남매 함께 입소 순함,True,[순함]
369,448532202000848,수호년월생추정 남매 함께 입소 순함,True,[순함]
